## 章節 4：更多的估計

### 4.4 效率最佳化

到目前做者寫的程式碼都是為了易於閱讀所寫，但是沒有效率。

總的來說，作者喜歡開發程式法確定正確信，接著確定是否效率符合這個的任務。如果可以接受，就不必要做最佳化。

例如硬幣問題，如果很關心效率，有很多地方可以加速。

#### 1. 減少正規化（Normalize）的次數

<pre>
dataset = 'H' * heads + 'T' * tails
for data in dataset: # 每次轉硬幣呼叫一次 Update 方法
    suite.Update(data)

        
def Update(self, data):
    for hypo in self.Values():
        like = self.Likelihood(data, hypo)
        self.Mult(hypo, like)
        
    return self.Normalize() # 呼叫一次 Update 便需要執行 Normalize 一次，每 Normalize 一次便需要在跑遍所有的假設組
</pre>

我們可以減少 Normalize 方法的呼叫次數。Suite 類別提供 UpdateSet 方法便可以減少呼叫次數。如下：

<pre>
def UpdateSet(self, dataset):
    for data in dataset:
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
            
    return self.Normalize() #整個資料集看過後，在做正規化
</pre>

使用的實作
<pre>
dataset = 'H' * heads + 'T' * tails
suite.UpdateSet(dataset)
</pre>

這的確有加速到，但執行時間任然與資料成正比。我們可以改寫 Likelihood 直接處理整個資料集，而不是一次看一筆資料。

#### 2. 改寫 Likelihood 方法

原始版本
<pre>
def Likelihood(self, data, hypo):
    x = hypo / 100.0
    if data == 'H':
        return x

    return 1-x
</pre>

加速版本，將資料用兩個數字分別表示正面以及反面出現的次數。

<pre>
def Likelihood(self, data, hypo):
    x = hypo / 100.0
    heads, tails = data
    like = x**heads * (1-x)**tails
    return like
</pre>

使用的實作
<pre>
heads, tails = 140, 110
suite.Update((heads, tails))
</pre>

將迴圈乘法直接用次方取代，這個版本的執行時間對於其他轉硬幣的次數都是一樣的。